In [1]:
import cv2
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm

# Загрузка предобученной модели MiDaS
model_type = "DPT_Large"  # Можно использовать "DPT_Hybrid" или "MiDaS_small"
model = torch.hub.load("intel-isl/MiDaS", model_type)
model.eval()

# Определение преобразований для входного изображения
transform = transforms.Compose([
    transforms.Resize(384),  # Измените размер в зависимости от модели
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def predict_depth(image):
    """Предсказание карты глубины для одного кадра"""
    # Преобразование изображения
    image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    input_tensor = transform(image_pil).unsqueeze(0)

    # Прогнозирование глубины
    with torch.no_grad():
        depth_map = model(input_tensor)

    # Нормализация карты глубины
    depth_map = depth_map.squeeze().cpu().numpy()
    depth_map = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min())

    return depth_map

def process_video(input_video_path, output_video_path):
    """Обработка видео и сохранение результата"""
    # Открытие видеофайла
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Ошибка открытия видеофайла")
        return

    # Получение параметров видео
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Создание VideoWriter для сохранения результата
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width * 2, height))

    # Обработка кадров с прогресс-баром
    progress_bar = tqdm(total=total_frames, desc="Обработка видео")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Предсказание карты глубины
        depth_map = predict_depth(frame)

        # Визуализация карты глубины
        depth_vis = (depth_map * 255).astype(np.uint8)
        depth_vis = cv2.applyColorMap(depth_vis, cv2.COLORMAP_JET)
        depth_vis = cv2.resize(depth_vis, (width, height))

        # Объединение оригинального кадра и карты глубины
        combined_frame = np.hstack((frame, depth_vis))

        # Запись кадра в выходной файл
        out.write(combined_frame)

        progress_bar.update(1)

    # Освобождение ресурсов
    cap.release()
    out.release()
    progress_bar.close()
    cv2.destroyAllWindows()

    print(f"Обработка завершена. Результат сохранен в {output_video_path}")

if __name__ == "__main__":
    input_video_path = "/content/1112645_Approach_Winter-clothes_3840x2160 (online-video-cutter.com).mp4"  # Укажите путь к исходному видео
    output_video_path = "output_depth_video.mp4"  # Путь для сохранения результата

    process_video(input_video_path, output_video_path)

/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/intel-isl/MiDaS/zipball/master" to /root/.cache/torch/hub/master.zip
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.l

Обработка завершена. Результат сохранен в output_depth_video.mp4
